In [1]:
import os
import torch
import torch.nn as nn
from tqdm.auto import tqdm
from tqdm import tqdm
import json
import math
import numpy as np
import pandas as pd
import random

/mnt/data/smart_health_02/fanyongqi/anaconda3/envs/chatglm_etuning/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM
from transformers import StoppingCriteriaList,GenerationConfig

In [3]:
#your model path or model name
pretrained_model_name_or_path = "Johnnyfans/normpulse"

In [4]:
tokenizer = AutoTokenizer.from_pretrained(
	pretrained_model_name_or_path = pretrained_model_name_or_path,
	padding_side='left',
)

model = AutoModelForCausalLM.from_pretrained(
	pretrained_model_name_or_path = pretrained_model_name_or_path,
	torch_dtype=torch.float16,
 	device_map = 'auto'
)

Loading checkpoint shards: 100%|██████████| 2/2 [00:36<00:00, 18.18s/it]


In [5]:
model = model.eval()

In [6]:
def eos_criteria(input_ids, scores=None, tokenizer=tokenizer):
    if input_ids.shape[0] != 1:
        return False
    if input_ids[0][-1] == tokenizer.eos_token_id:
        return True
    return False

def dict2cuda(dicts):
    dict_tmp = {}
    for i in dicts:
        dict_tmp[i] = dicts[i].cuda()
    return dict_tmp

In [7]:
generate_args = {
"min_new_tokens": 1,
"max_new_tokens": 128,
"early_stopping": True,
"num_beams": 1,
"do_sample": False,
"temperature": 1.0,
"top_k": 9,
"top_p": 0.7,
"num_return_sequences": 1,
}
gen_config = GenerationConfig(**generate_args)


In [8]:
def kc_predict(query, base_prompt, model, tokenizer, top_k=10, top_p=0.7, temperature=1.0, input_max_len=256, gen_max_length=256, do_sample=True):

    first_instruction = "Instructions: You are Helper, a large language model full of intelligence. Respond conversationally."
    
    query = query.strip() 

    input_ids = tokenizer(
        first_instruction,
        add_special_tokens=False
    ).input_ids + [tokenizer.convert_tokens_to_ids("</s>")]
    
    
    input_ids += tokenizer("User: " + base_prompt.replace('{x}',query), add_special_tokens=False).input_ids
    input_ids += [tokenizer.convert_tokens_to_ids("</s>")]  

    model_inputs = tokenizer.pad(
        {"input_ids": [input_ids + tokenizer("Helper: ", add_special_tokens=False).input_ids[:1]]}, 
        return_tensors="pt",
    )

    inputs = model_inputs.input_ids[:,-input_max_len:]
    attention_mask = model_inputs.attention_mask[:,-input_max_len:]

    max_length = inputs.shape[1] + gen_max_length
    min_length = inputs.shape[1] + 1 # add eos

    outputs = model.generate(
        inputs=inputs.cuda(),
        attention_mask=attention_mask.cuda(),
        max_length=max_length,
        min_length=min_length,
        do_sample=do_sample,
        top_k=top_k,
        top_p=top_p,
        temperature=temperature,
        num_return_sequences=1,
        eos_token_id=tokenizer.convert_tokens_to_ids("</s>"),
    )

    outputs_token = outputs[0].tolist()

    new_start_pos = inputs.shape[1]
    new_end_pos = new_start_pos

    while new_end_pos < len(outputs_token) and outputs_token[new_end_pos] != tokenizer.convert_tokens_to_ids("</s>"):
        new_end_pos += 1

    outputs_token = list(tokenizer("Helper: ", add_special_tokens=False).input_ids[:1]) + outputs_token[new_start_pos:new_end_pos]

    input_ids += outputs_token
    input_ids += [tokenizer.convert_tokens_to_ids("</s>")] 

    otext = tokenizer.decode(
        outputs_token, 
        skip_special_tokens=False
    )[8:]
    
    return otext

Knowledge cards generation examples

In [9]:
dis_kc_prompt = 'User：知识卡片生成，请依据你掌握的医疗知识，根据输入的疾病诊断术语生成它的知识卡片，包括它的定义描述、病因、病理、部位、疾病类型和临床表现（如症状、特征、分割、分类、性别、年龄、急性慢性、发病时间等）。{x}'
op_kc_prompt = 'User：知识卡片生成，请依据你掌握的医疗知识，根据输入手术操作术语生成它的知识卡片，包括它的定义描述、手术术式、作用部位、手术入路、手术疾病性质等信息。{x}'

In [10]:
kc_predict(query='一型糖尿病',base_prompt=dis_kc_prompt, model=model, tokenizer=tokenizer)

You're using a BloomTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


'定义描述：一型糖尿病是一种慢性代谢性疾病，由于胰岛素分泌不足或完全缺乏导致血糖水平持续升高。\n疾病类型：一型糖尿病属于胰岛素依赖型糖尿病，一般发生于儿童和青少年。\n部位：一型糖尿病影响整个身体的代谢过程，主要涉及胰岛和血液中的葡萄糖调节。\n病因：一型糖尿病的主要病因是自身免疫反应导致胰岛β细胞受损和破坏，减少或完全缺乏胰岛素的分泌。\n病理：一型糖尿病的病理特点是胰岛β细胞减少或消失，胰岛内淋巴细胞浸润，导致胰岛功能受损。\n临床表现：一型糖尿病的典型临床表现包括多饮、多食、多尿和体重减轻。患者可能出现疲劳、视力模糊、感染易发等其他症状。此外，一型糖尿病患者需要注射外源性胰岛素以控制血糖水平。\n请注意，以上信息基于一般知识，并非针对特定个体。如果您或他人有相关症状或疑虑，请咨询医生进行确诊和治疗。'

In [11]:
kc_predict(query='肩袖修补术',base_prompt=op_kc_prompt, model=model, tokenizer=tokenizer)

'定义描述：肩袖修补术是一种用于治疗肩关节病变的手术方法，旨在修复或替代肩袖肌腱，以恢复肩关节的稳定性和功能。\n手术术式：肩袖修补术通常包括肩袖肌腱缝合术或肌腱移植术，有时还伴有关节镜检查和关节镜手术。\n部位：肩部（肩关节区域）。\n病因：肩袖修补术多用于治疗肩关节退行性病变、肩袖肌腱损伤或撕裂等情况。常见病因包括肩袖肌腱炎症、退行性变、肩关节损伤和肩袖肌腱撕裂。\n病理：肩袖修补术并非针对特定病理类型，而是通过手术方法修复或替代肩袖肌腱，以改善肩关节功能和稳定。\n临床表现：肩袖修补术后，患者可能出现肩关节疼痛、僵硬、肩关节功能改善、肩部力量恢复以及肩关节活动范围受限等临床表现。具体表现因个体差异和手术方法不同而异。\n注意：以上信息仅为一般性描述，具体病情和手术细节应在专业医生指导下确定，并遵循医生的建议进行术后恢复和治疗'

Clinical Term Normalization examples

In [12]:
def norm_predict(query_list, base_prompt, model, tokenizer, top_k=10, top_p=0.7, temperature=1.0, input_max_len=256, gen_max_length=256, do_sample=True):

    first_instruction = "Instructions: You are Helper, a large language model full of intelligence. Respond conversationally."
    
    query = query_list[0]
    cand = '\n'.join(query_list[-1])
    
    query = query.strip() 

    input_ids = tokenizer(
        first_instruction,
        add_special_tokens=False
    ).input_ids + [tokenizer.convert_tokens_to_ids("</s>")]
    
    
    input_ids += tokenizer("User: " + base_prompt.replace('{mention}',query).replace('{cand}',cand), add_special_tokens=False).input_ids
    input_ids += [tokenizer.convert_tokens_to_ids("</s>")]  

    model_inputs = tokenizer.pad(
        {"input_ids": [input_ids + tokenizer("Helper: ", add_special_tokens=False).input_ids[:1]]}, 
        return_tensors="pt",
    )

    inputs = model_inputs.input_ids[:,-input_max_len:]
    attention_mask = model_inputs.attention_mask[:,-input_max_len:]

    max_length = inputs.shape[1] + gen_max_length
    min_length = inputs.shape[1] + 1 # add eos

    outputs = model.generate(
        inputs=inputs.cuda(),
        attention_mask=attention_mask.cuda(),
        max_length=max_length,
        min_length=min_length,
        do_sample=do_sample,
        top_k=top_k,
        top_p=top_p,
        temperature=temperature,
        num_return_sequences=1,
        eos_token_id=tokenizer.convert_tokens_to_ids("</s>"),
    )

    outputs_token = outputs[0].tolist()

    new_start_pos = inputs.shape[1]
    new_end_pos = new_start_pos

    while new_end_pos < len(outputs_token) and outputs_token[new_end_pos] != tokenizer.convert_tokens_to_ids("</s>"):
        new_end_pos += 1

    outputs_token = list(tokenizer("Helper: ", add_special_tokens=False).input_ids[:1]) + outputs_token[new_start_pos:new_end_pos]

    input_ids += outputs_token
    input_ids += [tokenizer.convert_tokens_to_ids("</s>")] 

    otext = tokenizer.decode(
        outputs_token, 
        skip_special_tokens=False
    )[8:]
    
    return otext

In [13]:
norm_prompt = '''请扮演一个医学专家，你需要从候选标准术语列表中找出输入常用术语对应的标准术语，注意对应的标准术语可能有多个。若找到对应的标准术语则输出这些标准术语，有多个标准术语则用'##'分隔；若找不到对应的标准术语则输出“无对应标准术语”，并推荐可能对应的标准术语。
注意：1. 保证输出的标准术语是否在候选术语列表中；2. 不要进行多余的解释和说明
常用术语：{mention}
候选标准术语列表：
{cand}
输出：
'''

In [14]:
mention = '肩袖修补术'

In [15]:
candidate= ['关节结构的修补术和整形术', '关节结构的切开术和切除术', '其他诊断性放射学和相关技术', '其他骨的切开术、切除术和切断术', '子宫和支持结构的其他手术', '心脏瓣膜和间隔手术', '手部肌、腱和筋膜手术', '淋巴系统手术', '眼眶和眼球手术', '肌、腱、筋膜和粘液囊手术', '肌肉骨骼系统的其他操作', '胸壁、胸膜、纵隔和横膈手术', '血管其他手术', '颅和周围神经的手术', '骨折和脱位复位术', '骨的其他手术']

In [16]:
norm_predict(query_list=[mention,candidate],base_prompt=norm_prompt, model=model, tokenizer=tokenizer, do_sample=False)

'肌、腱、筋膜和粘液囊手术'

In [17]:
candidate= ['肌、腱和筋膜的缝合术', '肌、腱和筋膜切断术', '肌、腱和筋膜其他整形术', '肌和腱重建术', '肌、腱和筋膜的其他切除术']

In [18]:
norm_predict(query_list=[mention,candidate],base_prompt=norm_prompt, model=model, tokenizer=tokenizer, do_sample=False)

'肌、腱和筋膜的缝合术'

In [19]:
candidate =  ['腱鞘缝合术', '腱延迟性缝合术', '回旋肌环带修补术', '腱的其他缝合术', '肌肉或筋膜的其他缝合术']

In [20]:
norm_predict(query_list=[mention,candidate],base_prompt=norm_prompt, model=model, tokenizer=tokenizer, do_sample=False)

'回旋肌环带修补术'

In [21]:
candidate =  ['腱鞘缝合术', '腱延迟性缝合术', '腱的其他缝合术', '肌肉或筋膜的其他缝合术']

In [22]:
norm_predict(query_list=[mention,candidate],base_prompt=norm_prompt, model=model, tokenizer=tokenizer, do_sample=False)

'无对应标准术语，根据当前候选标准术语列表的情况，推荐可能对应的标准词为：回旋肌环带修补术'